# Setup NVFLARE in POC Mode


Setting Up the Application Environment in POC Mode¶
>**Warning**
>POC mode is not intended to be secure and should not be run in any type of production environment or any environment where the server’s ports are exposed. For actual deployment and even development, it is recommended to use a secure provisioned setup or The FL Simulator.

To get started with a proof of concept (POC) setup after Installation, run this command to generate a poc folder with an overseer, server, two clients, and one admin client:


In [ ]:
!nvflare poc -h

First, we need to prepare a workspace. 
you can use
``` 
   nvflare poc --prepare -n N
```
or

```
nvflare poc --prepare 
```
for default number of client = 2

The default workspace = /tmp/nvflare/poc
If you prefer to set workspace in different location than the default, you set the environment variable 

```
export NVFLARE_POC_WORKSPACE = /you/new/location/path

```

before running the nvflare poc command. If you running the POC prepare command, there is a promot for user answer. we are going to pre-answer as Yes in the following

In [ ]:
! echo 'y' |  nvflare poc --prepare

Now we start the 2 clients and 1 server, but without FLARE console (aka Admin Console), as we are going to interact with the system from notebook

### Start Flare system in POC

Next we need create a terminal to execute the following command
create a terminal (in JupyterLab or host terminal), then

```
  nvflare poc --start -ex admin
```

In [ ]:
%%bash --bg  

nvflare poc --start -ex admin
echo "sleep for few seconds to let system start"
sleep 3

In [ ]:
! ps -eaf | grep nvflare

* Check System information

In [ ]:

import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(workspace, "admin")
sess = new_insecure_session(admin_dir)
sys_info = sess.get_system_info()

print(f"Server info:\n{sys_info.server_info}")
print("\nclient_info")
for client in sys_info.client_info:
    print(client)

Once the system is up. Now we are go back the notebook to work on your project. 

### Stop FLARE system in POC

Once you are done with the Flare system, you can shutdown the poc system.
From terminal, issue the following command stop poc in the terminal
```
   nvflare poc --stop
```


### Cleanup POC workspace

In [4]:
! nvflare poc --clean

/tmp/nvflare/poc is not valid poc directory
